In [1]:
# Import packages
import pandas as pd
import os 
from scipy.spatial.distance import cdist
from datetime import datetime 

In [2]:
# Get current directory
current_directory = os.getcwd()
date = datetime.now().strftime("%Y%m%d")
cutoff = 25
# Find datafile
path = f"{current_directory}/Data/distances/dists_cellpairs_{cutoff}px_{date}.csv"

# path = f"{current_directory}/Data/distances/dists_cellpairs_25px_20240229.csv"
dists = pd.read_csv(path)

In [18]:
dists

,source_ID,target_ID,distance,source_X,source_Y,target_X,target_Y,source_cluster,target_cluster,Patient_ID
0,D205_1442,D205_0,11.180340,109,368,107,379,Cancer,Cancer,LUAD_D205
1,D205_1471,D205_0,13.416408,95,385,107,379,Cancer,Cancer,LUAD_D205
2,D205_1475,D205_0,20.000000,95,395,107,379,Cancer,Cancer,LUAD_D205
3,D205_2079,D205_0,12.041595,95,378,107,379,Cancer,Cancer,LUAD_D205
4,D205_2096,D205_0,20.000000,91,367,107,379,Cancer,Cancer,LUAD_D205
...,...,...,...,...,...,...,...,...,...,...
17104169,D395_157,D395_999,19.235384,372,212,375,231,Cancer,Cancer,LUAD_D395
17104170,D395_162,D395_999,15.620499,385,219,375,231,Endothelial cell,Cancer,LUAD_D395
17104171,D395_1736,D395_999,25.000000,368,207,375,231,Cancer,Cancer,LUAD_D395
17104172,D395_2321,D395_999,13.038405,376,218,375,231,Cancer,Cancer,LUAD_D395


In [6]:
dists['source_ID'].nunique()

1637722

In [4]:
# Convert datafile of distances to relative frequency of source and target cell per source_id and per patient
count_df = dists.groupby(['Patient_ID','source_ID','source_cluster', 'target_cluster']).size().reset_index(name='count')
total_counts_scid = count_df.groupby(['Patient_ID','source_ID','source_cluster'])['count'].sum().reset_index(name='total_scid')

count_df

,Patient_ID,source_ID,source_cluster,target_cluster,count
0,LUAD_D001,D001_0,Cancer,Alt MAC,1
1,LUAD_D001,D001_0,Cancer,Cancer,1
2,LUAD_D001,D001_0,Cancer,Cl MAC,1
3,LUAD_D001,D001_0,Cancer,Tc,1
4,LUAD_D001,D001_0,Cancer,Th,2
...,...,...,...,...,...
5051531,LUAD_D416,D416_998,Cl MAC,Tc,1
5051532,LUAD_D416,D416_998,Cl MAC,Th,2
5051533,LUAD_D416,D416_999,Cancer,Cancer,5
5051534,LUAD_D416,D416_999,Cancer,Cl MAC,2


In [12]:
dists['source_ID'].nunique()

1637722

In [13]:
count_df['source_ID'].nunique()

1507024

In [ ]:


count_df = pd.merge(count_df, total_counts_scid, on=['Patient_ID','source_ID' ,'source_cluster'])
totalscid = count_df['total_scid']
# print(count_df.head())
count_df['relative_frequency'] = (count_df['count'] / count_df['total_scid'])

# Convert dataframe to correct format for clustering
sc = dists['source_cluster'].unique()
freq = count_df.pivot(columns=['target_cluster'], index=['Patient_ID','source_ID','source_cluster'], values=['relative_frequency'])
freq.reset_index(inplace=True)
freq.columns  = freq.columns.droplevel(0)
freq.columns = ['Patient_ID','source_ID','source_cluster',*freq.columns[3:]]
freq.fillna(0, inplace=True)

print('Dataset converted, below is a preview:\n',freq.head())

# Save file
output_path = f"{current_directory}/Data/frequencies/"

# totalscid.to_csv(f"{output_path}frequencies_cells_{cutoff}px_{date}.csv", index = False)
freq.to_csv(f"{output_path}frequencies_{cutoff}px_{date}.csv", index = False)